In [ ]:
import os
import sys

project_dir = os.path.realpath(os.path.join(os.getcwd(), '..','..','..','..'))
tool_dir = os.path.join(project_dir, 'Utils')
sys.path.append(tool_dir)

In [ ]:
import pandas as pd
from workers import *
from MRS import regimeSwitch, mrsPreprocess, get_MRS_stats
import warnings
import numpy as np

plt.style.use('seaborn-whitegrid')
warnings.filterwarnings("ignore")

In [ ]:
start = '2019-01-01'
end = '2021-05-14'
ticker = ['QQQ']
df = get_all_price(ticker, start, end)
df.head()

In [ ]:
processed_data = mrsPreprocess(df)
train, test = processed_data.split_train_test(testNum=80)

train_endog, train_exog = train['ret'], train[['lag1_ret','lag2_ret']]
test_endog, test_exog = test['ret'], test[['lag1_ret','lag2_ret']]

In [ ]:
predictV, predictP, rmse, actual, model = get_MRS_stats(train_endog, train_exog, test_endog, test_exog)
rmsedf = pd.DataFrame(rmse)
rmsedf

In [ ]:
rmsedf.T.plot(figsize=(12,7),title='RMSE for 2 models',kind='bar')

In [ ]:
regimeSwitch.plotCompareModel(actual, predictV, 'Out-of-sample Predicted vs Actual Stock Return')

In [ ]:
predictP[0].plot(title='QQQ Predicted Probability of 3 regimes under model 0', figsize=(10,5))
predictP[1].plot(title='QQQ Predicted Probability of 3 regimes under model 1', figsize=(10,5))

In [ ]:
plt.figure(figsize=(8,5))
actual.plot(title='QQQ stock return ',color='k')
mrsPreprocess(df).df['ret'].plot(linestyle='dotted')